# Load data

In [53]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D

In [54]:
import pandas as pd
import numpy as np

label2idx = {'무증상' : 0, '결막염' : 1, '궤양성각막질환' : 2 , '백내장' : 3 , '비궤양성각막질환' : 4,
               '색소침착성각막염' : 5, '안검내반증' : 6, '안검염' : 7, '안검종양' : 8, '유루증' : 9, '핵경화' : 10}
idx2label = {0: '무증상', 1: '결막염', 2: '궤양성각막질환', 3: '백내장', 4: '비궤양성각막질환',
               5: '색소침착성각막염', 6: '안검내반증', 7: '안검염', 8: '안검종양', 9: '유루증', 10: '핵경화'}

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
base_load = '/content/drive/MyDrive/image_recognition/data/sample/t_u'
base_load_m = '/content/drive/MyDrive/image_recognition/data/sample/t_m'

img_u1 = pd.read_pickle(base_load + '/결막염_images_224_200.pickle')
img_u2 = pd.read_pickle(base_load + '/궤양성각막질환_images_224_200.pickle')
img_u3 = pd.read_pickle(base_load + '/백내장_images_224_200.pickle')
img_u4 = pd.read_pickle(base_load + '/비궤양성각막질환_images_224_200.pickle')
img_u5 = pd.read_pickle(base_load + '/핵경화_images_224_200.pickle')

img_m1 = pd.read_pickle(base_load_m + '/결막염_images_224_100.pickle')
img_m2 = pd.read_pickle(base_load_m + '/궤양성각막질환_images_224_100.pickle')
img_m3 = pd.read_pickle(base_load_m + '/백내장_images_224_100.pickle')
img_m4 = pd.read_pickle(base_load_m + '/비궤양성각막질환_images_224_100.pickle')
img_m5 = pd.read_pickle(base_load_m + '/핵경화_images_224_100.pickle')


print(img_u1.shape, img_u2.shape, img_u3.shape, img_u4.shape, img_u5.shape) 
print(img_m1.shape, img_m2.shape, img_m3.shape, img_m4.shape, img_m5.shape)

(100, 224, 224, 3) (200, 224, 224, 3) (300, 224, 224, 3) (200, 224, 224, 3) (100, 224, 224, 3)
(100, 224, 224, 3) (100, 224, 224, 3) (100, 224, 224, 3) (100, 224, 224, 3) (100, 224, 224, 3)


In [73]:
# concat
img_con1 = np.concatenate((img_u1,img_m1))
img_con2 = np.concatenate((img_u2,img_m2))
img_con3 = np.concatenate((img_u3,img_m3))
img_con4 = np.concatenate((img_u4,img_m4))
img_con5 = np.concatenate((img_u5,img_m5))

In [74]:
# label
label1 = np.concatenate((np.full(100, 1),np.full(100, 0)))
label2 = np.concatenate((np.full(100, 1),np.full(100, 0)))
label3 = np.concatenate((np.full(100, 1),np.full(100, 0)))
label4 = np.concatenate((np.full(100, 1),np.full(100, 0)))
label5 = np.concatenate((np.full(100, 1),np.full(100, 0)))

In [75]:
from sklearn.model_selection import train_test_split

# train_test_split
train_images1, test_images1, train_labels1, test_labels1 = train_test_split(img_u1, label1, stratify = label1 , test_size=0.2)
train_images2, test_images2, train_labels2, test_labels2 = train_test_split(img_m2, label2, stratify = label2 , test_size=0.2)
train_images3, test_images3, train_labels3, test_labels3 = train_test_split(img_m3, label3, stratify = label3 , test_size=0.2)
train_images4, test_images4, train_labels4, test_labels4 = train_test_split(img_m4, label4, stratify = label4 , test_size=0.2)
train_images5, test_images5, train_labels5, test_labels5 = train_test_split(img_m5, label5, stratify = label5 , test_size=0.2)

ValueError: ignored

In [ ]:
# print(img_con5.shape)

#  Load model

### 결막염

In [20]:
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

In [ ]:
train_images1.shape, test_images1.shape, train_labels1.shape, test_labels1.shape

In [ ]:
import cv2
plt.imshow(cv2.cvtColor(s_train_images[3], cv2.COLOR_BGR2RGB))

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50


## 일부 재학습 모델
base_model_50 = ResNet50(include_top=False, input_shape = (224,224 ,3), weights = 'imagenet')

base_model_50.trainable = True	

for layer in base_model_50.layers[:-10]:
    layer.trainable = False		

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model_50(inputs, training=False) # batchnorm 부분 update 방지

x = tf.keras.layers.Flatten(input_shape=base_model_50.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
s_model_res50 = tf.keras.Model(inputs, outputs)
